# Introduction

We will build from scratch a class of convolutional neural networks (CNNs) for 2D, implementing the algorithms using only minimal libraries such as NumPy.


We will also create a pooling layer and so on to complete the basic form of the CNN. The name of the class should be Scratch2dCNNClassifier.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def println(*str):
    for i in str:
        print(i)

# Data Set

In [3]:
#data set
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# #reshape
# X_train = X_train.reshape(-1, 784)
# X_test = X_test.reshape(-1, 784)
#scaling
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
#one hot encode for multiclass labels!
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
#validation split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, train_size=0.5)

# Suppporting Classes

In [4]:
from numpy.random import default_rng
rng = default_rng()

class SimpleInitializer:
    def __init__(self, sigma = 0.1):
        self.sigma = sigma
    
    def W(self, dimension = ()):
        return rng.normal(0, self.sigma, dimension)
    
    def B(self,n=0):
        return rng.normal(0, self.sigma, (1,n))


# Test
simp_init = SimpleInitializer()
w = simp_init.W(dimension = (3,2))
b = simp_init.B(2)

println('W', w)
println('B', b)

x = np.random.randint(5,size = (2,3))
println('X', x)

println('x*w + b', x@w + b)

W
[[-0.01827949 -0.09281002]
 [ 0.04515009  0.0752372 ]
 [-0.17195084 -0.02320633]]
B
[[0.02203956 0.152103  ]]
X
[[3 3 1]
 [4 2 2]]
x*w + b
[[-0.06929949  0.07617823]
 [-0.3046799  -0.11507532]]


# Problem 1
2D Convolutional Layer

In [5]:
class Conv2d:
    def __init__(self, initializer = SimpleInitializer(), kernel_size = (3,3), n_out_channels = 3, padding = 0, stride = 1):
        #! let's not consider padding and stride at the moment
        
        self.initializer = initializer
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.n_out_channels = n_out_channels


    def forward(self, X,y, X_val = 0, y_val = 0):
        if X.shape.ndim > 4:
            raise 'do not support 4 dimension array'
        
        # init size
        self.n_input, self.n_row, self.n_col, self.n_in_channels = X.shape

        # init weight and biases
        self.W = self.initializer.W(dimension = (self.n_in_channels, self.n_out_channels, *self.kernel_size))
        pass

    def backward(self, y):
        pass
    

# Test Forward Propagation

In [13]:
# creating data, weight and bias
np.random.seed(0)
n_in = 2
n_out = 3
dim = (2,3)
kernel_size = (2,2)
print('n_in', n_in)
print('n_out', n_out)
print('dim',dim)
print('kernel_size', kernel_size)

X = np.random.randint(0,10,(n_in, *dim))
println('X',X.shape,  X)

W = np.random.randint(0,2,(n_out, n_in, *kernel_size)) #init kernel
B = np.random.randint(0,1, n_out)
println('W', W.shape, W)
println('B', B)

n_in 2
n_out 3
dim (2, 3)
kernel_size (2, 2)
X
(2, 2, 3)
[[[5 0 3]
  [3 7 9]]

 [[3 5 2]
  [4 7 6]]]
W
(3, 2, 2, 2)
[[[[0 0]
   [0 0]]

  [[1 0]
   [1 1]]]


 [[[0 0]
   [1 1]]

  [[1 1]
   [0 1]]]


 [[[0 1]
   [0 1]]

  [[1 0]
   [1 1]]]]
B
[0 0 0]


In [14]:
# duplicate X to match n_out
X = np.vstack([[X]] * n_out)
println('X', X.shape, X)

X
(3, 2, 2, 3)
[[[[5 0 3]
   [3 7 9]]

  [[3 5 2]
   [4 7 6]]]


 [[[5 0 3]
   [3 7 9]]

  [[3 5 2]
   [4 7 6]]]


 [[[5 0 3]
   [3 7 9]]

  [[3 5 2]
   [4 7 6]]]]


In [17]:
# forward

in_x, in_y = dim
ker_x, ker_y = kernel_size
conv_size_x = in_x - ker_x + 1
conv_size_y = in_y - ker_y + 1
output_shape = (conv_size_x, conv_size_y)
result = np.ones((n_out, *output_shape))
print('expected output shape: ', output_shape)
print('result shape: ', result.shape)
print('Xshape: ', X.shape)
for i in range(conv_size_x):
    for j in range(conv_size_y):
        print('convolving at: ', i,j, 'ker: ', ker_x, ker_y)
        temp_x = X[:,:, i : i + ker_x, j : j + ker_y]
        # println('before: ', temp_x, temp_x.shape)
        temp = temp_x * W
        # print(1,temp)
        temp = np.sum(temp, axis = (2,3))
        # print(2, temp)
        temp = np.sum(temp, axis = 1) 
        # print(3,temp)
        result[:,i,j] = temp + B
        print(4,result[:,i,j] )

println('Forward', result.shape, result)

expected output shape:  (1, 2)
result shape:  (3, 1, 2)
Xshape:  (3, 2, 2, 3)
convolving at:  0 0 ker:  2 2
4 [14. 25. 21.]
convolving at:  0 1 ker:  2 2
4 [18. 29. 30.]
Forward
(3, 1, 2)
[[[14. 18.]]

 [[25. 29.]]

 [[21. 30.]]]


(2, 2, 2)


(3, 2, 2, 2)